# Pedicle Screws Planner
> my projection Notebook

## 导入

In [ ]:
#export
from __main__ import qt, ctk, vtk, slicer

from .PedicleScrewSimulatorStep import *
from .Helper import *
import DICOM
import PythonQt
import numpy as np
import string
import math
import os
import time
import logging
import csv
import datetime
import re

## 创建类

In [ ]:
class LoadDataStep(PedicleScrewSimulatorStep):

    def __init__( self, stepid ):
      self.initialize( stepid )
      slicer.util.setDataProbeVisible(False)
      self.setName('Pedicle Screw Planner: One Click One Level')
      self.setDescription(
        """Select a screw;\nIn the Red Slice or the Yellow Slice, Drag both ends of the analog screw to adjust the length and angle;\nUpdata;\nSelect the Diameter of the screw";\nAfter reaching the ideal size and angle,Click OK Generate the Screw""")
      # """1.Place the Crosshair in the center of ROI and then click the "ROI"bottom;\n2. In 3D slice viewer, if the front  """

      # 加载图像之后,将crosshair放置到想要的ROI的中心,然后点击"ROI"按钮获取ROI;在3d视窗里观察椎体前壁,如果不完整(由于OP等原因),调节最小阈值,直到满意(忽略那些渣子);点击"Create",生成"ROI Model";将crosshair移动到上终板(同时按shift键),调整横断面(PTA)平行于上终板(同时按shift+cmd或者crtl),调整矢状面(PSA)(按键同上)使得crosshair的红线居于骨性椎管最大横径,绿线居于椎体的正中面,点击"oneClick",等待...,生成虚拟的Pedicle Screw;不满意?进入Adjust Screw, 选择直径,或者手动调节角度和长度,点击"updata",直到满意;最后点击"OK"生成Pedicle Screw Model,或者你会需要导出数据可以点击"save".


      self.__vrDisplayNode = None
      self.__threshold = [ -1, -1 ]
          
      # initialize VR stuff
      self.__vrLogic = slicer.modules.volumerendering.logic()
      self.__vrOpacityMap = None
      self.fidlist = []
      self.dimeter = []
      self.length = []
      self.PSA = []
      self.PTA = []
      self.Cdata = None
      self.levels = (
        "C1", "C2", "C3", "C4", "C5", "C6", "C7", "T1", "T2", "T3", "T4", "T5", "T6", "T7", "T8", "T9", "T10", "T11",
        "T12",
        "L1", "L2", "L3", "L4", "L5", "S1")  # , "place")
      self.buttonToModName = ''
      self.__parent = super( LoadDataStep, self )

### 杀按钮

In [ ]:
#export
def killButton(self):
      # hide useless button
      bl = slicer.util.findChildren(text='Final')
      if len(bl):
        bl[0].hide()

### 聚焦

In [ ]:
#export
def zoomIn(self):
      logging.debug("zoom")
      slicer.app.applicationLogic().PropagateVolumeSelection(1)

### 界面

In [ ]:
#export
def createUserInterface( self ):

      self.__layout = self.__parent.createUserInterface()
     
      # self.fid = Helper.Screws()
      # self.fids = len(self.fid)
      self.fidlist = ["Choose the puncture level"]
      self.dimeter = []
      self.length = []
      self.PSA = []
      self.PTA = []
      self.CZn = 0
      self.CXn = 0
      self.CLstep = 0
      self.CData = 3.5
      self.cScrew = ""
      self.csNo = 0
      self.screwList = []
      self.manulYn = 0
      self.gradeIsthu = []

      blank = qt.QLabel(">>>>>>>>>>>>>>>")

      self.stepButtonGroup = qt.QButtonGroup()
      self.stepButtonGroup.setExclusive(True)

      self.__roiFrame = ctk.ctkCollapsibleButton()
      self.__roiFrame.text = "Define ROI and Segmentation"
      self.__roiFrame.collapsed = False
      roiFrameLayout = qt.QFormLayout(self.__roiFrame)
      self.__layout.addRow(self.__roiFrame)
      self.stepButtonGroup.addButton(self.__roiFrame)

      oText = qt.QLabel('ROI&segmentation:')
      self.cropROIButton = qt.QPushButton('ROI')
      self.notROIButton = qt.QPushButton('Reset')
      self.cropROIButton.connect('clicked(bool)', self.cropROI)
      self.notROIButton.connect('clicked(bool)', self.notROI)

      self.QHBox = qt.QHBoxLayout()
      self.QHBox.addWidget(oText)
      self.QHBox.addWidget(self.cropROIButton)
      self.QHBox.addWidget(self.notROIButton)
      # self.QHBox9.addWidget(self.okButton)
      # self.__layout.addRow(self.QHBox)
      roiFrameLayout.addRow(self.QHBox)

      threshLabel = qt.QLabel('Choose threshold:')
    # threshold value
      self.__minThresholdSlider = ctk.ctkSliderWidget()
      self.__minThresholdSlider.singleStep = 10
      self.__minThresholdSlider.minimum = 0
      self.__minThresholdSlider.maximum = 300
      self.__minThresholdSlider.value = 150
      self.__minThresholdSlider.setToolTip("set the minThreshold to adjust the model.")
      self.__minThresholdSlider.connect('valueChanged(double)', self.onThresholdChanged)

      self.QHBox = qt.QHBoxLayout()
      self.QHBox.addWidget(threshLabel)
      self.QHBox.addWidget(self.__minThresholdSlider)
      # self.QHBox9.addWidget(self.okButton)
      # self.__layout.addRow(self.QHBox)
      roiFrameLayout.addRow(self.QHBox)

      modText = qt.QLabel('creat model:')
      self.modCreatButton = qt.QPushButton('OK')
      self.modDelButton = qt.QPushButton('Reset')
      self.modCreatButton.connect('clicked(bool)', self.modCreat)
      self.modDelButton.connect('clicked(bool)', self.modDel)

      self.QHBox4 = qt.QHBoxLayout()
      self.QHBox4.addWidget(modText)
      self.QHBox4.addWidget(self.modCreatButton)
      self.QHBox4.addWidget(self.modDelButton)
      roiFrameLayout.addRow(self.QHBox4)

      self.__oneClickFrame = ctk.ctkCollapsibleButton()
      self.__oneClickFrame.text = "One Click One Level"
      self.__oneClickFrame.collapsed = True
      oneClickFrameLayout = qt.QFormLayout(self.__oneClickFrame)
      self.__layout.addRow(self.__oneClickFrame)
      self.stepButtonGroup.addButton(self.__oneClickFrame)

      sText = qt.QLabel('Choose the puncture level:')
      self.sSelector = ctk.ctkComboBox()
      self.sSelector.toolTip = "Choose the puncture level"
      screwList = ['Choose the puncture site', "L1", "L2", "L3", "L4", "L5"]
      self.sSelector.addItems(screwList)
      # self.sSelector.addItems(self.fidlist)
      self.connect(self.sSelector, PythonQt.QtCore.SIGNAL('activated(QString)'), self.sSelector_chosen)
      self.__sSelector = ''
      onText = qt.QLabel('One Click One Level:')
      self.oneClickButton = qt.QPushButton("oneClick")
      self.oneClickButton.connect('clicked(bool)', self.oneClickok)
      self.notoneClickButton = qt.QPushButton("Reset")
      self.notoneClickButton.connect('clicked(bool)', self.notoneClickok)

      self.QHBoxS = qt.QHBoxLayout()
      self.QHBoxS.addWidget(sText)
      self.QHBoxS.addWidget(self.sSelector)
      oneClickFrameLayout.addRow(self.QHBoxS)

      self.QHBox0 = qt.QHBoxLayout()
      self.QHBox0.addWidget(onText)
      self.QHBox0.addWidget(self.oneClickButton)
      self.QHBox0.addWidget(self.notoneClickButton)
      oneClickFrameLayout.addRow(self.QHBox0)

      self.__adjustFrame = ctk.ctkCollapsibleButton()
      self.__adjustFrame.text = "Adjust Screw"
      self.__adjustFrame.collapsed = True
      adjustFrameLayout = qt.QFormLayout(self.__adjustFrame)
      self.__layout.addRow(self.__adjustFrame)
      self.stepButtonGroup.addButton(self.__adjustFrame)

      lText = qt.QLabel("Current Level in ")
      self.sideSelector = ctk.ctkComboBox()
      sidesList = ['Left or Right','Left', 'Right']
      self.sideSelector.addItems(sidesList)
      self.connect(self.sideSelector, PythonQt.QtCore.SIGNAL('activated(QString)'), self.sideSelector_chosen)
      self.__sideSelector = ''

      self.QHBox2 = qt.QHBoxLayout()
      self.QHBox2.addWidget(lText)
      self.QHBox2.addWidget(self.sideSelector)
      adjustFrameLayout.addRow(self.QHBox2)

      dText = qt.QLabel('Select screw diameter:')
      self.dSelector = ctk.ctkComboBox()
      self.dSelector.toolTip = "Select screw diameter"
      dimList = ['Select screw diametermm', "2.5", "3", "3.5", "4", "4.5", "5", "5.5", "6", "6.5", "7", "7.5", "8"]
      self.dSelector.addItems(dimList)
      self.connect(self.dSelector, PythonQt.QtCore.SIGNAL('activated(QString)'), self.dSelector_chosen)
      self.__dSelector = ''
      mText = qt.QLabel('Manual Adjust:')
      self.manButton = qt.QPushButton("Update")
      self.manButton.connect('clicked(bool)', self.manualUp)

      self.QHBox3 = qt.QHBoxLayout()
      self.QHBox3.addWidget(dText)
      self.QHBox3.addWidget(self.dSelector)
      # self.QHBox3.addWidget(self.manButton)
      adjustFrameLayout.addRow(self.QHBox3)

      self.QHBox3 = qt.QHBoxLayout()
      self.QHBox3.addWidget(mText)
      # self.QHBox3.addWidget(self.dSelector)
      self.QHBox3.addWidget(self.manButton)
      adjustFrameLayout.addRow(self.QHBox3)

      self.QHBoxbl = qt.QHBoxLayout()
      self.QHBoxbl.addWidget(blank)
      self.__layout.addRow(self.QHBoxbl)

      cText = qt.QLabel('Creat Screw Model:')
      self.okButton = qt.QPushButton("OK")
      self.resetButton = qt.QPushButton("Reset")
      self.okButton.connect('clicked(bool)', self.okShow)
      self.resetButton.connect('clicked(bool)', self.reset)

      self.QHBox9 = qt.QHBoxLayout()
      self.QHBox9.addWidget(cText)
      self.QHBox9.addWidget(self.okButton)
      self.QHBox9.addWidget(self.resetButton)
      self.__layout.addRow(self.QHBox9)

      exText = qt.QLabel('Export Data:')
      self.saveButton = qt.QPushButton('Save As .csv')
      self.saveButton.connect('clicked(bool)', self.exportData)
      # self.previewButton = qt.QPushButton('Preview Tables')
      # self.previewButton.connect('clicked(bool)', self.previewTable)


      self.QHBoxex = qt.QHBoxLayout()
      self.QHBoxex.addWidget(exText)
      self.QHBoxex.addWidget(self.saveButton)
      # self.QHBoxex.addWidget(self.previewButton)
      self.__layout.addRow(self.QHBoxex)





      #Active Volume text
      self.activeText = qt.QLabel("Spine CT:")
      self.__layout.addRow(self.activeText)
      #select volume
      #creates combobox and populates it with all vtkMRMLScalarVolumeNodes in the scene
      self.__inputSelector = slicer.qMRMLNodeComboBox()
      self.__inputSelector.nodeTypes = ( ("vtkMRMLScalarVolumeNode"), "" )
      self.__inputSelector.addEnabled = False
      self.__inputSelector.removeEnabled = False
      self.__inputSelector.setMRMLScene( slicer.mrmlScene )
      self.__layout.addRow(self.__inputSelector )

      self.items = []

      qt.QTimer.singleShot(0, self.killButton)

      transform = slicer.vtkMRMLLinearTransformNode()
      transform.SetName("Camera Transform")
      slicer.mrmlScene.AddNode(transform)

      cam = slicer.mrmlScene.GetNodeByID('vtkMRMLCameraNode1')
      cam.SetAndObserveTransformNodeID('vtkMRMLLinearTransformNode4')    

### 德芙  

InitVRDisplayNode(self)

In [ ]:
#export

    def InitVRDisplayNode(self)
#

      # if self.__vrDisplayNode == None:
      #   pNode = self.parameterNode()
      #   vrNodeID = pNode.GetParameter('vrDisplayNodeID')
      #   if vrNodeID == '':
      #     self.__vrDisplayNode = slicer.modules.volumerendering.logic().CreateVolumeRenderingDisplayNode()
      #     v = slicer.mrmlScene.GetNodeByID(self.parameterNode().GetParameter('baselineVolumeID'))
      #     self.__vrDisplayNode = Helper.InitVRDisplayNode(v.GetID(), self.__roi.GetID())
      #   else:
      #     self.__vrDisplayNode = slicer.mrmlScene.GetNodeByID(vrNodeID)

      slicer.modules.volumerendering.logic().CopyDisplayToVolumeRenderingDisplayNode(self.__vrDisplayNode)

      self.__vrOpacityMap = self.__vrDisplayNode.GetVolumePropertyNode().GetVolumeProperty().GetScalarOpacity()
      self.__vrColorMap = self.__vrDisplayNode.GetVolumePropertyNode().GetVolumeProperty().GetRGBTransferFunction()

      # setup color transfer function once
      self.__vrColorMap.RemoveAllPoints()
      self.__vrColorMap.AddRGBPoint(0, 0.95,0.84,0.57)
      self.__vrColorMap.AddRGBPoint(500, 0.95,0.84,0.57)
      # Update transfer function based on ROI
      self.processROIEvents()

    # def onROIChanged(self):
    #   # read ROI node from combobox
    #   roi = self.__roiSelector.currentNode()

    #   if roi != None:
    #     self.__roi = roi

    #     # create VR node first time a valid ROI is selected
    #     self.InitVRDisplayNode()

    #     # update VR settings each time ROI changes
    #     pNode = self.parameterNode()
    #     # get scalar volume node loaded in previous step
    #     v = pNode.GetNodeReference('baselineVolume')

    #     # set parameters for VR display node
    #     self.__vrDisplayNode.SetAndObserveROINodeID(roi.GetID())
    #     self.__vrDisplayNode.SetCroppingEnabled(1)
    #     self.__vrDisplayNode.VisibilityOn()

    #     # transform ROI
    #     # roi.SetAndObserveTransformNodeID(self.__roiTransformNode.GetID())
        

    #     if self.__roiObserverTag != None:
    #       self.__roi.RemoveObserver(self.__roiObserverTag)

    #     # add observer to ROI. call self.processROIEvents if ROI is altered
    #     self.__roiObserverTag = self.__roi.AddObserver('ModifiedEvent', self.processROIEvents)

    #     # enable click and drag functions on ROI
    #     roi.SetInteractiveMode(1)

    #     # connect ROI widget to ROI
    #     self.__roiWidget.setMRMLAnnotationROINode(roi)
    #     self.__roi.SetDisplayVisibility(1)
    


### 德芙  

onThresholdChanged(self,value)

In [ ]:
#export

    def onThresholdChanged(self,value)
#
 
      
      if self.__vrOpacityMap == None:
        return
      
      self.thresholdMin = value
      self.thresholdMax = 2000
      # logging.debug("min:{}".format(range0))

      self.__vrOpacityMap.RemoveAllPoints()
      self.__vrOpacityMap.AddPoint(0,0)
      self.__vrOpacityMap.AddPoint(0,0)
      self.__vrOpacityMap.AddPoint(self.thresholdMin-1,0)
      self.__vrOpacityMap.AddPoint(self.thresholdMin,1)
      self.__vrOpacityMap.AddPoint(self.thresholdMax,1)
      self.__vrOpacityMap.AddPoint(self.thresholdMax+1,0)

      # update the label volume accordingly
      thresh = vtk.vtkImageThreshold()
      thresh.SetInputData(self.__croppedROI.GetImageData())
      thresh.ThresholdBetween(self.thresholdMin, self.thresholdMax)
      thresh.SetInValue(1)
      thresh.SetOutValue(0)
      thresh.ReplaceOutOn()
      thresh.ReplaceInOn()
      thresh.Update()

      self.__roiSegmentationNode.SetAndObserveImageData(thresh.GetOutput())
      Helper.SetBgFgVolumes(self.__croppedROI.GetID())


### 德芙  

cameraFocus(self, position, distance= -30)

In [ ]:
#export

    def cameraFocus(self, position, distance= -30)
#

      camera = slicer.mrmlScene.GetNodeByID('vtkMRMLCameraNode1')
      camera.SetFocalPoint(*position)
      camera.SetPosition(position[0], distance, position[2])
      camera.SetViewUp([0, 0, 1])
      camera.ResetClippingRange()


### 德芙  

sliceFocus(self, side)

In [ ]:
#export

    def sliceFocus(self, side)
#


      if side == "Left":
        Pz = self.Pl
        PB = self.PBl
      elif side == "Right":
        Pz = self.Pr
        PB = self.PBr
      else:
        print("side: Left or Right")
        return

      self.redSlicerData = [Pz, PB, [Pz[0], PB[1], PB[2]]]
      self.yellowSlicerData = [Pz, PB, [PB[0], PB[1], Pz[2]]]
      self.cameraFocus(PB)
      Helper.UpdateSlicePlane(self.redSlicerData, "Red")
      Helper.UpdateSlicePlane(self.yellowSlicerData, "Yellow", 2, 1)
      # slicer.util.setSliceViewerLayers(self.vol, fit=True)


### 德芙  

cropROI(self)

In [ ]:
#export

    def cropROI(self)
#

      PP = Helper.getPlaneIntersectionPoint()  
      volumeNode = slicer.util.getNode("vtkMRMLScalarVolumeNode*")
    # Create a new ROI 新建ROI
      self.__roi = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLAnnotationROINode")
    # Make new parameter node in order to use the crop volume module programmatically
    # 制作新的参数节点，以便以编程方式使用cropVolume模块
      crop_module = slicer.vtkMRMLCropVolumeParametersNode()
      # Add parameter node to the scene 给场景添加参数节点
      slicer.mrmlScene.AddNode(crop_module)
      # Set the volume as the input volume in the crop volume module 设置输入卷
      crop_module.SetInputVolumeNodeID(volumeNode.GetID())
      # Set output volume as the same volume to overwrite original volume (only needed if you actually want to crop the volume)
    #   cropped.SetOutputVolumeNodeID(volumeNode.GetID())
      # Set the input ROI 设置输入ROI
      crop_module.SetROINodeID(self.__roi.GetID())
      # Use the Fit ROI to Volume function of the crop volume module 用ROI框卷
      slicer.modules.cropvolume.logic().FitROIToInputVolume(crop_module)
      # 设置ROI
      self.__roi.SetName("ROI")
      self.__roi.SetXYZ(PP)
      self.__roi.SetRadiusXYZ(50, 50, 100)
      cropLogic = slicer.modules.cropvolume.logic()
      # cvpn = slicer.vtkMRMLCropVolumeParametersNode()
      crop_module.SetROINodeID( self.__roi.GetID() )
      cropLogic.Apply( crop_module )
      # 获得cropvolume的输出结果
      self.__croppedROI = slicer.mrmlScene.GetNodeByID( crop_module.GetOutputVolumeNodeID() )
      self.__croppedROI.SetName("croppedROI")
      volRenLogic = slicer.modules.volumerendering.logic()
      vl = slicer.modules.volumes.logic()
      # 新建segmentation
      self.__roiSegmentationNode = vl.CreateAndAddLabelVolume(slicer.mrmlScene, self.__croppedROI, 'croppedROI_segmentation')
      # 渲染节点
      self.__vrDisplayNode = volRenLogic.CreateDefaultVolumeRenderingNodes(self.__croppedROI)
      # 对比度节点
      self.__vrOpacityMap =self.__vrDisplayNode.GetVolumePropertyNode().GetVolumeProperty().GetScalarOpacity()
      # 颜色节点
      self.__vrColorMap =self.__vrDisplayNode.GetVolumePropertyNode().GetVolumeProperty().GetRGBTransferFunction()
    # self.__baselineROIVolume = slicer.util.getNode("baselineROI")
      # ROI维度
      baselineROIRange = self.__croppedROI.GetImageData().GetScalarRange()
      # 加色
      self.__vrColorMap.RemoveAllPoints()
      # self.__vrColorMap.AddRGBPoint(0, 0, 0, 0) 
      # self.__vrColorMap.AddRGBPoint(baselineROIRange[0]-1, 0, 0, 0) 
      self.__vrColorMap.AddRGBPoint(baselineROIRange[0],0.95,0.84,0.57)
      self.__vrColorMap.AddRGBPoint(baselineROIRange[1], 0.95,0.84,0.57)
      # self.__vrColorMap.AddRGBPoint(baselineROIRange[1]+1, 0, 0, 0)

      # self.__vrDisplayNode.VisibilityOn()

      # threshRange = [150, 2000]
      # self.__vrOpacityMap.RemoveAllPoints()
      # self.__vrOpacityMap.AddPoint(0,0)
      # self.__vrOpacityMap.AddPoint(0,0)
      # self.__vrOpacityMap.AddPoint(threshRange[0]-1,0)
      # self.__vrOpacityMap.AddPoint(threshRange[0],1)
      # self.__vrOpacityMap.AddPoint(threshRange[1],1)
      # self.__vrOpacityMap.AddPoint(threshRange[1]+1,0)

      labelsColorNode = slicer.modules.colors.logic().GetColorTableNodeID(10)
      self.__roiSegmentationNode.GetDisplayNode().SetAndObserveColorNodeID(labelsColorNode)
      Helper.SetLabelVolume(self.__roiSegmentationNode.GetID())

      self.onThresholdChanged(150)

      # self.__roiSegmentationNode.SetAndObserveImageData(thresh.GetOsutput())
      # Update transfer function based on ROI
      # self.processROIEvents()
      
      return 



### 德芙  

notROI(self)

In [ ]:
#export

    def notROI(self)
#

      try:
        Helper.delNode("AnnotationROI*")
        Helper.delNode("ROI")
        Helper.delNode("croppedROI*")
      finally:
        pNode = self.parameterNode()
        self.__croppedROI = pNode.GetNodeReference('croppedROI')
        Helper.SetBgFgVolumes(self.__croppedROI.GetID())
        Helper.SetLabelVolume(None)


### 德芙  

modCreat(self)

In [ ]:
#export

    def modCreat(self)
#

      Helper.skinSur(MiniThresd=self.thresholdMin)
      self.__vrDisplayNode.VisibilityOff()
      Helper.delNode("Model*")
    

### 德芙  

modDel(self)

In [ ]:
#export

    def modDel(self)
#

      Helper.delNode("boneTempt")
      self.__vrDisplayNode.VisibilityOn()



### 德芙  

sSelector_chosen(self, text)

In [ ]:
#export

    def sSelector_chosen(self, text)
#


      if text != "Choose the puncture site":
        self.__sSelector = text


### 德芙  

oneClickok(self)

In [ ]:
#export

    def oneClickok(self)
#

      self.data = Helper.oneClicks(self.__sSelector)
      sHiera = slicer.util.getNode("SubjectHierarchy")
      self.__patientName =  re.findall("\\n    Name: (.+?)\\n",str(sHiera))[0]
      self.Pl = self.data[0][0]
      self.Pr = self.data[0][1]
      self.PBl = self.data[1][0]
      self.PBr = self.data[1][1]
      # date = datetime.datetime.now()
      # datestr = date.strftime('%m-%d-%H-%M')
      # datarr = self.data[2]
      # datalist = [self.__sSelector] + datarr.tolist() + [datestr]
      # filepath = r"/Users/liguimei/Docments/finish/jingtriangleingtee/data/data/"

      # with open(r"{}data.csv".format(filepath), "a", newline='') as f:
      #   writer = csv.writer(f)
      #   writer.writerow(datalist)
      # f.close()
      # logging.debug("data[3]:{}".format(self.data[3]))
      # np.savetxt(r'{}grade_{}_l_{}.csv'.format(filepath, self.__sSelector, datestr), self.data[3], delimiter=",")
      # logging.debug("data[4]:{}".format(self.data[4]))
      # np.savetxt(r'{}grade_{}_r_{}.csv'.format(filepath, self.__sSelector, datestr), self.data[4], delimiter=",")
      # logging.debug("fileexportdone")
      # with open("data/data.txt", "a", newline='') as f:
      #   f.write("{},{}".format(self.__sSelector,str(self.data)))
      #   f.write("\n")
      #   f.close()
      self.screwList.append([self.__patientName, self.__sSelector, self.data[2]])
      self.gradeIsthu.append([self.__patientName, self.__sSelector, self.data[3], self.data[4]])
      logging.debug("{}:".format(self.screwList))
      logging.debug("{}:".format(self.gradeIsthu))


### 德芙  

notoneClickok(self)

In [ ]:
#export

    def notoneClickok(self)
#

      pass


### 德芙  

sideSelector_chosen(self, text)

In [ ]:
#export

    def sideSelector_chosen(self, text)
#

      if text != "Choose the Current Side":
        self.__sideSelector = text
        self.currentFidIndex = "Ln_{}_{}".format(self.__sSelector, text)
        self.sliceFocus(self.__sideSelector)


### 德芙  

dSelector_chosen(self, text)

In [ ]:
#export

    def dSelector_chosen(self, text)
#


      if text != "Select the diameter of the screw":
        self.__dSelector = float(text)
        self.Cdata = Helper.Screw(self.currentFidIndex, self.Pz, self.__dSelector)
        logging.debug("self.currentFidIndex:{}".format(self.currentFidIndex))


### 德芙  

manualUp(self)

In [ ]:
#export

    def manualUp(self)
#

      self.Cdata = Helper.Screw(self.currentFidIndex, self.Pz, manulYN=True)
      logging.debug("self.currentFidIndex:{}".format(self.currentFidIndex))


### 德芙  

okShow12(self)

In [ ]:
#export

    def okShow12(self)
#

      import datetime
      date = datetime.datetime.now()
      datestr = date.strftime('%m-%d-%H')
      # slicer.util.saveScene(r"{0}/{0}.mrb".format(datestr))
      screwlist = str(self.screwList)
      # logging.debug("self.screwList:{}".format(screwlist))
      with open(r"/Users/liguimei/Docments/finish/jingtriangleingtee/data/data.csv", "a", newline='') as f:
        f.write(screwlist)
        f.write("{}\n".format(datestr))
        f.close()
      gradeIsthu = str(self.gradeIsthu)
      # logging.debug("self.screwList:{}".format(gradeIsthu))
      with open(r"/Users/liguimei/Docments/finish/jingtriangleingtee/data/dataIsthu.csv", "a", newline='') as f:
        f.write(gradeIsthu)
        f.write("{}\n".format(datestr))
        f.close()
      logging.debug("fileexportdone")


### 德芙  

okShow(self)

In [ ]:
#export

    def okShow(self)
#

      lineNode = "Ln_{}*".format(self.currentFidIndex)
      lineN = Helper.Psline(lineNode)
      PB = lineN[0]
      PT = lineN[1]
      Dim = lineN[3]
      # Length = lineN[3]
      markupsNode = slicer.util.getNode(lineNode)
      markupsNode.CreateDefaultDisplayNodes()
      dn = markupsNode.GetDisplayNode()
      dn.SetGlyphTypeFromString("CrossDot2D")
      dn.SetGlyphScale(1)
      # Helper.delNode(lineNode)
      screwName = "Screw_{}".format(self.__sSelector)
      # Pz0 = Helper.p2pexLine(PB,self.Pz,10)
      # Helper.p2pCyl(Pz0,self.Pz)
      PB_Pz = np.linalg.norm(PT - PB) * (self.Pz[1] - PB[1]) / (PT[1] - PB[1])
      logging.debug("PB_Pz:{}".format(PB_Pz))
      PZ = Helper.p2pexLine(PB, PT, 5 + PB_Pz - np.linalg.norm(PT - PB))
      # PZ0 = Helper.p2pexLine(PB,PT,PB_Pz-np.linalg.norm(PT-PB))
      # Helper.addFid(PZ)
      # Helper.addFid(PZ0)
      Helper.p2pCyl(PZ, PB, Dim * .5, "Isthmus_{}".format(screwName), 5 - PB_Pz, 12, "white", .5)
      Helper.p2pCyl(PB, PT, Dim * .5, screwName, 0, 12, "white", .25)
      screwAngle = Helper.screwAngle(self.currentFidIndex, PZ)
      logging.debug("screwAngle:{}".format(screwAngle))
      if self.Cdata is None:
        self.screwList.append(
          [self.__sSelector, 90 - screwAngle[0], screwAngle[1], self.cScrewData[0], self.cScrewData[1],
           self.cScrewData[2], self.cScrewData[3], self.Pz])
      else:
        self.screwList.append(
          [self.__sSelector, 90 - screwAngle[0], screwAngle[1], self.Cdata[0], self.Cdata[1], self.Cdata[2],
           self.Cdata[3], self.Pz])


### 德芙  

reset(self)

In [ ]:
#export

    def reset(self)
#

      self.Pz = self.CData[1]
      self.Pa = self.CData[0]
      self.cScrewData = Helper.Screw(self.currentFidIndex, self.Pz)
      # pass
    

### 德芙  

onApplyButton(self)

In [ ]:
#export

    def onApplyButton(self)
#

      # file picker
      qfd = qt.QFileDialog()
      qfd.windowTitle = 'Save Output As'
      qfd.modal = True
      qfd.setFilter('Comma-separated values (*.csv)')
      qfd.acceptMode = qt.QFileDialog.AcceptSave
      qfd.fileMode = qt.QFileDialog.AnyFile
      qfd.defaultSuffix = 'csv'
      qfd.selectFile('data.csv')
      
      if qfd.exec_() == qt.QFileDialog.AcceptSave:
        fname = qfd.selectedFiles()[0]      
        logic = HistogramLogic()
        logic.run(self.inputSelector.currentNode(), self.imask.currentNode(), fname=fname, binrange=(float(self.range_min.text), float(self.range_max.text)), bins=int(self.bins.text), byframe=self.byframe_box.isChecked()) 


### 德芙  

onEntry(self, comingFrom, transitionType)

In [ ]:
#export

    def onEntry(self, comingFrom, transitionType)
#


      super(LoadDataStep, self).onEntry(comingFrom, transitionType)

      # setup the interface
      lm = slicer.app.layoutManager()
      lm.setLayout(3)
      viewNodes = slicer.util.getNodesByClass('vtkMRMLSliceCompositeNode')
      for viewNode in viewNodes:
        viewNode.SetSliceIntersectionVisibility(1)

      # roiTransformNode = pNode.GetNodeReference('roiTransform')
      # if not roiTransformNode:
      #   roiTransformNode = slicer.vtkMRMLLinearTransformNode()
      #   slicer.mrmlScene.AddNode(roiTransformNode)
      #   pNode.SetNodeReferenceID('roiTransform', roiTransformNode.GetID())

      # dm = vtk.vtkMatrix4x4()
      # self.__baselineVolume.GetIJKToRASDirectionMatrix(dm)
      # dm.SetElement(0,3,0)
      # dm.SetElement(1,3,0)
      # dm.SetElement(2,3,0)
      # dm.SetElement(0,0,abs(dm.GetElement(0,0)))
      # dm.SetElement(1,1,abs(dm.GetElement(1,1)))
      # dm.SetElement(2,2,abs(dm.GetElement(2,2)))
      # roiTransformNode.SetMatrixTransformToParent(dm)

      qt.QTimer.singleShot(0, self.killButton)


    #check that conditions have been met before proceeding to next step #在进行下一步之前检查条件是否已满足

### 德芙  

validate( self, desiredBranchId )

In [ ]:
#export

    def validate( self, desiredBranchId )
#


      self.__parent.validate( desiredBranchId )

      #read current scalar volume node
      self.__baseline = self.__inputSelector.currentNode()

      #if scalar volume exists proceed to next step and save node ID as 'baselineVolume'
      pNode = self.parameterNode()
      if self.__baseline:
        baselineID = self.__baseline.GetID()
        if baselineID:
          pNode = self.parameterNode()
          pNode.SetNodeReferenceID('baselineVolume', baselineID)
          self.__parent.validationSucceeded(desiredBranchId)
      else:
        self.__parent.validationFailed(desiredBranchId, 'Error','Please load a volume before proceeding')

    #called when exiting step

### 德芙  

onExit(self, goingTo, transitionType)

In [ ]:
#export

    def onExit(self, goingTo, transitionType)
#


      #check to make sure going to correct step
      if goingTo.id() == 'DefineROI':
        self.doStepProcessing();

      if goingTo.id() != 'DefineROI':
        return

      super(LoadDataStep, self).onExit(goingTo, transitionType)

    # def processROIEvents(self,node=None,event=None):
    #   # get the range of intensities inside the ROI

    #   # Make updates faster and prevent flickering (due to transfer function editing)
    #   slicer.app.pauseRender()

    #   # get the IJK bounding box of the voxels inside ROI
    #   roiCenter = [0,0,0]
    #   roiRadius = [0,0,0]

    #   #get center coordinate
    #   self.__roi.GetXYZ(roiCenter)
    #   logging.debug(roiCenter)

    #   #change slices to center of ROI
    #   lm = slicer.app.layoutManager()
    #   redWidget = lm.sliceWidget('Red')
    #   redController = redWidget.sliceController()

    #   yellowWidget = lm.sliceWidget('Yellow')
    #   yellowController = yellowWidget.sliceController()

    #   greenWidget = lm.sliceWidget('Green')
    #   greenController = greenWidget.sliceController()

    #   yellowController.setSliceOffsetValue(roiCenter[0])
    #   greenController.setSliceOffsetValue(roiCenter[1])
    #   redController.setSliceOffsetValue(roiCenter[2])

    #   #get radius
    #   self.__roi.GetRadiusXYZ(roiRadius)

    #   #get IJK coordinates of 8 corners of ROI
    #   roiCorner1 = [roiCenter[0]+roiRadius[0],roiCenter[1]+roiRadius[1],roiCenter[2]+roiRadius[2],1]
    #   roiCorner2 = [roiCenter[0]+roiRadius[0],roiCenter[1]+roiRadius[1],roiCenter[2]-roiRadius[2],1]
    #   roiCorner3 = [roiCenter[0]+roiRadius[0],roiCenter[1]-roiRadius[1],roiCenter[2]+roiRadius[2],1]
    #   roiCorner4 = [roiCenter[0]+roiRadius[0],roiCenter[1]-roiRadius[1],roiCenter[2]-roiRadius[2],1]
    #   roiCorner5 = [roiCenter[0]-roiRadius[0],roiCenter[1]+roiRadius[1],roiCenter[2]+roiRadius[2],1]
    #   roiCorner6 = [roiCenter[0]-roiRadius[0],roiCenter[1]+roiRadius[1],roiCenter[2]-roiRadius[2],1]
    #   roiCorner7 = [roiCenter[0]-roiRadius[0],roiCenter[1]-roiRadius[1],roiCenter[2]+roiRadius[2],1]
    #   roiCorner8 = [roiCenter[0]-roiRadius[0],roiCenter[1]-roiRadius[1],roiCenter[2]-roiRadius[2],1]

    #   #get RAS transformation matrix of scalar volume and convert it to IJK matrix
    #   ras2ijk = vtk.vtkMatrix4x4()
    #   self.__croppedROI.GetRASToIJKMatrix(ras2ijk)

    #   roiCorner1ijk = ras2ijk.MultiplyPoint(roiCorner1)
    #   roiCorner2ijk = ras2ijk.MultiplyPoint(roiCorner2)
    #   roiCorner3ijk = ras2ijk.MultiplyPoint(roiCorner3)
    #   roiCorner4ijk = ras2ijk.MultiplyPoint(roiCorner4)
    #   roiCorner5ijk = ras2ijk.MultiplyPoint(roiCorner5)
    #   roiCorner6ijk = ras2ijk.MultiplyPoint(roiCorner6)
    #   roiCorner7ijk = ras2ijk.MultiplyPoint(roiCorner7)
    #   roiCorner8ijk = ras2ijk.MultiplyPoint(roiCorner8)

    #   lowerIJK = [0, 0, 0]
    #   upperIJK = [0, 0, 0]

    #   lowerIJK[0] = min(roiCorner1ijk[0],roiCorner2ijk[0],roiCorner3ijk[0],roiCorner4ijk[0],roiCorner5ijk[0],roiCorner6ijk[0],roiCorner7ijk[0],roiCorner8ijk[0])
    #   lowerIJK[1] = min(roiCorner1ijk[1],roiCorner2ijk[1],roiCorner3ijk[1],roiCorner4ijk[1],roiCorner5ijk[1],roiCorner6ijk[1],roiCorner7ijk[1],roiCorner8ijk[1])
    #   lowerIJK[2] = min(roiCorner1ijk[2],roiCorner2ijk[2],roiCorner3ijk[2],roiCorner4ijk[2],roiCorner5ijk[2],roiCorner6ijk[2],roiCorner7ijk[2],roiCorner8ijk[2])

    #   upperIJK[0] = max(roiCorner1ijk[0],roiCorner2ijk[0],roiCorner3ijk[0],roiCorner4ijk[0],roiCorner5ijk[0],roiCorner6ijk[0],roiCorner7ijk[0],roiCorner8ijk[0])
    #   upperIJK[1] = max(roiCorner1ijk[1],roiCorner2ijk[1],roiCorner3ijk[1],roiCorner4ijk[1],roiCorner5ijk[1],roiCorner6ijk[1],roiCorner7ijk[1],roiCorner8ijk[1])
    #   upperIJK[2] = max(roiCorner1ijk[2],roiCorner2ijk[2],roiCorner3ijk[2],roiCorner4ijk[2],roiCorner5ijk[2],roiCorner6ijk[2],roiCorner7ijk[2],roiCorner8ijk[2])

    #   #get image data of scalar volume
    #   image = self.__croppedROI.GetImageData()

    #   #create image clipper
    #   clipper = vtk.vtkImageClip()
    #   clipper.ClipDataOn()
    #   clipper.SetOutputWholeExtent(int(lowerIJK[0]),int(upperIJK[0]),int(lowerIJK[1]),int(upperIJK[1]),int(lowerIJK[2]),int(upperIJK[2]))
    #   clipper.SetInputData(image)
    #   clipper.Update()

    #   #read upper and lower threshold values from clipped volume
    #   roiImageRegion = clipper.GetOutput()
    #   intRange = roiImageRegion.GetScalarRange()
    #   lThresh = 0.4*(intRange[0]+intRange[1])
    #   uThresh = intRange[1]

    #   #create new opacity map with voxels falling between upper and lower threshold values at 100% opacity. All others at 0%
    #   self.__vrOpacityMap.RemoveAllPoints()
    #   self.__vrOpacityMap.AddPoint(0,0)
    #   self.__vrOpacityMap.AddPoint(lThresh-1,0)
    #   self.__vrOpacityMap.AddPoint(lThresh,1)
    #   self.__vrOpacityMap.AddPoint(uThresh,1)
    #   self.__vrOpacityMap.AddPoint(uThresh+1,0)

    #   # finally, update the focal point to be the center of ROI
    #   camera = slicer.mrmlScene.GetNodeByID('vtkMRMLCameraNode1')
    #   camera.SetFocalPoint(roiCenter)
    #   camera.SetPosition(roiCenter[0],-600,roiCenter[2])
    #   camera.SetViewUp([0,0,1])

    #   slicer.app.resumeRender()


### 德芙  

doStepProcessing(self)

In [ ]:
#export

    def doStepProcessing(self)
#

      logging.debug('resultList:{}'.format(self.screwList))
      logging.debug('Done')

 s